In [2]:
import ffmpeg
import dateutil
import datetime

path = r"N:\Projekte\BASt_Wirksamkeit_Wildwarnanlagen_1754\Daten\Feldarbeit\Aglasterhausen B292\Waermebildkameras\session 1\wwa_B292_W_1_1\20210628\11\20210628_112010_8E7A_ACCC8EAE987D\20210628_11\20210628_113011_678E.mkv"
meta = ffmpeg.probe(path)
width = int(meta['streams'][0]['width'])
height = int(meta['streams'][0]['height'])
fps = int(meta['streams'][0]['avg_frame_rate'].split('/')[0])
duration = float(meta['format']['duration'])
starttime = dateutil.parser.isoparse(meta['format']['tags']['creation_time'])
endtime = starttime + datetime.timedelta(seconds=duration)
print(width, height, fps, starttime, endtime, duration)

720 576 15 2021-06-28 09:30:11.887511+00:00 2021-06-28 09:35:12.664700+00:00 300.777189


## Ergebnis der dritten Iteration der Klasse "Project"

In [1]:
import settings
from camtrappy.io.parsing import Project

p = Project(projectfolder=r'N:\Projekte\BASt_Wirksamkeit_Wildwarnanlagen_1754',
            folderstructure=r'Daten\Feldarbeit\{location}\Waermebildkameras\{session}\{sublocation}',
            videoformat='mkv')

kwargs = {'location': 'Aglasterhausen B292', 'session': 'session 1'}

videos = p.get_videos(**kwargs)
videos = list(videos)
print('Number of Videos:', len(videos))
print('First Video:', videos[0])


home:      n:\Projekte\Datenmanagement\Projekte\Wärmebildkamera-Analyse\CamTrapPy
data:      n:\Projekte\Datenmanagement\Projekte\Wärmebildkamera-Analyse\CamTrapPy\tests\data
notebooks: n:\Projekte\Datenmanagement\Projekte\Wärmebildkamera-Analyse\CamTrapPy\notebooks
--Setup complete--
Number of Videos: 10108
First Video: VideoFile(path='N:\\Projekte\\BASt_Wirksamkeit_Wildwarnanlagen_1754\\Daten\\Feldarbeit\\Aglasterhausen B292\\Waermebildkameras\\session 1\\wwa_B292_W_1_2\\20210716\\09\\20210716_092659_89CA_ACCC8EA768A4\\20210719_03\\20210719_033752_056B.mkv', location='Aglasterhausen B292', sublocation='wwa_B292_W_1_2', session='session 1', camera_no=None)


## Zweite Iteration für die Klasse "Project"

In [106]:
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional
    

@dataclass
class Project:
    projectfolder: str
    camptrappy_subfolder: str  # where to store data, configs, etc.
    locations_subfolder: Optional[str]  # contains all folders that are named after the locations
    videodata_subfolder: Optional[str]  # only necessary, if locationfolders contain other data as well
    with_sessions: bool  # are the recordings grouped into session folders?
    with_sublocations: bool  # were multiple cameras deployed at one location?
    sessions_before_sublocations: bool  # do sessions contain sublocations or do sublocations contain sessions?
    videoformat: str  # e.g. 'mkv'
    
    def __post_init__(self):
        if self.locations_subfolder is None: self.locations_subfolder = ''
        if self.videodata_subfolder is None: self.videodata_subfolder = ''
    
    def locations(self) -> List[str]:
        path = Path(self.projectfolder) / Path(self.locations_subfolder)
        locations = [p.name for p in path.glob('*/') if p.is_dir()]
        return locations

    def datafolder(self, location: str) -> str:
        path = Path(self.projectfolder) / Path(self.locations_subfolder) / location / Path(self.videodata_subfolder)
        return str(path)

    def videos(self):
        from itertools import chain
        
        locations = self.locations()
        lst = []
        for location in locations:
            videos = self.videos_by_location(location)
            lst.append(videos)
            
        return chain(*lst)

    def videos_by_location(self, location: str):
        path = Path(self.datafolder(location))
        videos = path.rglob(f'*.{self.videoformat}')

        for video in videos:
            
            if self.with_sessions or self.with_sublocations:
                subfolders = video.relative_to(path).parts
                
                ses, sub, ses_first = self.with_sessions, self.with_sublocations, self.sessions_before_sublocations
                
                if ses and sub and ses_first:
                    session, sublocation = subfolders[0], subfolders[1]
                elif ses and sub and not ses_first:
                    session, sublocation = subfolders[1], subfolders[0]
                elif ses and not sub:
                    session, sublocation = subfolders[0], None
                elif not ses and sub:
                    session, sublocation = None, subfolders[0]
                else:
                    session, sublocation = None, None
        
            else:
                session, sublocation = None, None
            
            yield(VideoFile(str(video), location, session, sublocation))

In [126]:
p = Project(r"N:\Projekte\BASt_Wirksamkeit_Wildwarnanlagen_1754", '', 'Daten/Feldarbeit', 'Waermebildkameras', True, True, True, 'mkv')
videos = p.videos()
x = list(videos)
len(x)

70120

In [74]:
for i in range(1000):
    next(videos)
next(videos)

VideoFile(path='N:\\Projekte\\BASt_Wirksamkeit_Wildwarnanlagen_1754\\Daten\\Feldarbeit\\Aglasterhausen B292\\Waermebildkameras\\session 1\\wwa_B292_W_2_1\\20210702\\08\\20210702_080325_EE85_ACCC8EA7680E\\20210703_00\\20210703_003107_9CE8.mkv', location='Aglasterhausen B292', session='session 1', sublocation='wwa_B292_W_2_1')

In [127]:
path = Path(r"N:\Projekte\BASt_Wirksamkeit_Wildwarnanlagen_1754")
files = path.rglob('*.mkv')
_ = list(files)
len(_)

70125